## Review: easy steps to train a world-class image classifier

1. Enable data augmentation, and precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
  * 3x -> original training images not super similar to target set
  * 10x -> original images are similar to target set
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting
1. Test Time Augmentation

In [43]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [7]:
PATH = "data/dogscats/"

In [4]:
sz = 224

In [31]:
!rm -rf {PATH}tmp

In [33]:
arch = resnet34

In [34]:
tfms = tfms_from_model(arch, sz)

## Training w/o data augmentation

In [35]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)

In [36]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 32/32 [00:07<00:00,  4.05it/s]


In [37]:
lrf = learn.lr_find()

learn.sched.plot_lr()
learn.sched.plot()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

 79%|███████▉  | 284/360 [00:04<00:01, 60.96it/s, loss=0.508]


In [41]:
learn.fit(1e-2, 3)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.04125  0.0228   0.9917 ]                         
[ 1.       0.04033  0.02109  0.99219]                         
[ 2.       0.0348   0.02074  0.99268]                         



## Training w/ Data Augmentation

In [44]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [46]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)

In [47]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [48]:
learn.fit(1e-2, 1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.05002  0.02897  0.98877]                         



In [49]:
learn.precompute=False

In [50]:
learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.04801  0.02583  0.98975]                         
[ 1.       0.04529  0.02569  0.99121]                         
[ 2.       0.04645  0.02574  0.99023]                         



In [51]:
learn.save('224_lastlayer_2')

In [52]:
learn.load('224_lastlayer_2')

## Train all layers w/ differential learning rates

In [55]:
learn.unfreeze()

In [57]:
lr = np.array([1e-4,1e-3,1e-2])

In [58]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.04447  0.02359  0.9917 ]                         
[ 1.       0.03975  0.02216  0.9917 ]                         
[ 2.       0.03215  0.02055  0.9917 ]                         
[ 3.       0.03498  0.02002  0.99268]                         
[ 4.       0.02284  0.01963  0.99219]                         
[ 5.       0.01991  0.01839  0.99268]                         
[ 6.       0.0181   0.01855  0.99219]                         



In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy(probs,y)

In [ ]:
learn.save('224_all_2')
learn.load('224_all_2')